In [132]:
import pandas as pd
from dataAug.tools import dataAugForTestSMOTE, dataAugForTestBorderlineSMOTE
from processTools import processRealData
import torch
from classify_model import ModelClassify
from targeTools import testThresholdFive, Accuracy
from dataAug.tools import dataAugForTestRandomOverSampler, dataAugForTestADASYN

In [133]:
feature_pd = pd.read_csv('/home/kongge/projects/new_protT5/data/ACC_T5_578.csv')
labels_pd = pd.read_csv("/home/kongge/projects/new_protT5/data/mutil_label_578.csv")

In [136]:

smote_multiple = {}
for item in range(1, 5, 2):
    G_feature, G_label = dataAugForTestSMOTE(feature_pd, labels_pd, item, 1044)
    R_feature, R_label = processRealData(feature_pd, labels_pd, feature_num=1044)

    train_feature = pd.concat([R_feature, G_feature], axis=0)
    train_label = pd.concat([R_label, G_label], axis=0)

    test_df = pd.read_csv("/home/kongge/projects/new_protT5/data/test_mutil_label_122_finally.csv")

    from torch.utils.data import TensorDataset, DataLoader
    datasetTrain = TensorDataset(torch.tensor(train_feature.values), torch.tensor(train_label.values))
    batch_size = len(train_feature)
    shuffle = True
    dataloaderTrain = DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

    datasetTest = TensorDataset(torch.tensor(test_df.iloc[:,:1044].values), torch.tensor(test_df.iloc[:,-4:].values))
    # 创建 DataLoader 对象，设置批量大小和是否打乱数据
    batch_size = len(datasetTest)
    dataloaderTest = DataLoader(datasetTest, batch_size=batch_size, shuffle=False)

    model = ModelClassify()
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0001)


    import warnings
    warnings.filterwarnings("ignore")
    GMList = {}
    for epoch in range(200):
        model.train()
        total_loss = 0.0
        for idx, data in enumerate(dataloaderTrain, 0):
            inputs, labels = data
            labels = labels.float()
            inputs = inputs.float()
            out = model(inputs)
            loss = criterion(out, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloaderTrain)
        threshold = 0.5
        labels_cov = torch.where(out > threshold, torch.tensor(1), torch.tensor(0))
        print(f"Epoch [{epoch+1}/{300}], Average Loss: {avg_loss:.4f}, ACC: {Accuracy(labels.int(), labels_cov)}")
        GMScore1 = testThresholdFive(epoch, model, dataloaderTest)
        GMList[epoch] = GMScore1

    best_key = max(GMList, key=GMList.get)
    best_value = GMList[best_key]
    smote_multiple[item] = best_value

Epoch [1/300], Average Loss: 0.7610, ACC: 0.20448838669177655
epoch:0,bestThreshold:[0.05, 0.52, 0.49, 0.05], GM:0.42415302991867065, OAA:0.0, ACC:0.32650273224043724, F1:0.484153151512146
Epoch [2/300], Average Loss: 0.5339, ACC: 0.570359907930531
epoch:1,bestThreshold:[0.05, 0.52, 0.05, 0.05], GM:0.41573771834373474, OAA:0.0, ACC:0.3101092896174864, F1:0.4666667878627777
Epoch [3/300], Average Loss: 0.4612, ACC: 0.6552626072400092
epoch:2,bestThreshold:[0.05, 0.52, 0.05, 0.49], GM:0.41704922914505005, OAA:0.0, ACC:0.3121584699453553, F1:0.4691258668899536
Epoch [4/300], Average Loss: 0.4115, ACC: 0.7176187486921952
epoch:3,bestThreshold:[0.05, 0.52, 0.48, 0.05], GM:0.4176503121852875, OAA:0.0, ACC:0.319672131147541, F1:0.4734974801540375
Epoch [5/300], Average Loss: 0.3646, ACC: 0.7937853107344632
epoch:4,bestThreshold:[0.05, 0.52, 0.47, 0.5], GM:0.438114732503891, OAA:0.040983606557377046, ACC:0.3613387978142079, F1:0.5040985345840454
Epoch [6/300], Average Loss: 0.3284, ACC: 0.8410

In [137]:
smote_multiple

{1: [tensor(0.7754),
  0.7213114754098361,
  0.7718579234972677,
  tensor(0.7951),
  tensor(0.7992),
  tensor(0.7896)],
 3: [tensor(0.7656),
  0.680327868852459,
  0.7595628415300546,
  tensor(0.7814),
  tensor(0.8197),
  tensor(0.7869)]}

In [2]:
import numpy as np

# 创建一个示例的 100 * 4 的 tensor
tensor = np.random.randint(0, 2, size=(100, 4))  # 假设原始数据是随机生成的 0 和 1

# 遍历每一行，检查是否所有元素都为0，如果是则随机设置一个元素为1


# 现在 tensor 中不会再出现所有元素都为0的行


In [3]:
tensor

array([[1, 0, 1, 1],
       [0, 1, 1, 0],
       [0, 0, 0, 0],
       [0, 1, 1, 0],
       [1, 1, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 1],
       [1, 0, 1, 0],
       [0, 1, 1, 1],
       [0, 0, 1, 1],
       [1, 1, 0, 1],
       [1, 0, 0, 0],
       [1, 1, 0, 1],
       [0, 1, 1, 0],
       [1, 1, 0, 1],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 0, 0],
       [1, 1, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [1, 0, 1, 1],
       [1, 0, 0, 0],
       [0, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 1, 1, 1],
       [0, 1, 1, 1],
       [0, 0, 1, 1],
       [1, 1, 0, 1],
       [0, 0, 0, 0],
       [1, 0, 1, 0],
       [1, 1, 0, 0],
       [1, 0, 1, 1],
       [1, 1, 0, 0],
       [1, 0, 1, 1],
       [1, 1, 0, 0],
       [0, 1, 1, 1],
       [1, 0, 1, 1],
       [0, 1, 1, 1],
       [0, 0, 1, 1],
       [0, 0,

In [4]:
for i in range(tensor.shape[0]):
    if np.all(tensor[i] == 0):
        random_index = np.random.randint(0, tensor.shape[1])
        tensor[i, random_index] = 1

In [5]:
tensor

array([[1, 0, 1, 1],
       [0, 1, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 1, 0],
       [1, 1, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 1],
       [1, 0, 1, 0],
       [0, 1, 1, 1],
       [0, 0, 1, 1],
       [1, 1, 0, 1],
       [1, 0, 0, 0],
       [1, 1, 0, 1],
       [0, 1, 1, 0],
       [1, 1, 0, 1],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 0, 0],
       [1, 1, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [1, 0, 1, 1],
       [1, 0, 0, 0],
       [0, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 1, 1, 1],
       [0, 1, 1, 1],
       [0, 0, 1, 1],
       [1, 1, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 1, 0],
       [1, 1, 0, 0],
       [1, 0, 1, 1],
       [1, 1, 0, 0],
       [1, 0, 1, 1],
       [1, 1, 0, 0],
       [0, 1, 1, 1],
       [1, 0, 1, 1],
       [0, 1, 1, 1],
       [0, 0, 1, 1],
       [0, 0,